In [1]:
using Flux
using DifferentialEquations
using DiffEqFlux
using LinearAlgebra
using JLD2
using Plots
using Statistics

using Flux: @epochs

using CuArrays
using CUDAdrv; CUDAdrv.name(CuDevice(0))

"GeForce GTX 960"

In [ ]:
file = jldopen("../data/ocean_convection_profiles.jld2");

In [ ]:
Is = keys(file["timeseries/t"])

Nz = file["grid/Nz"]
Nt = length(Is)

t = zeros(Nt)
T = T_data = zeros(Nt, Nz)

for (i, I) in enumerate(Is)
    t[i] = file["timeseries/t/$I"]
    T[i, :] = file["timeseries/T/$I"][1, 1, 2:Nz+1]
end

In [ ]:
z = file["grid/zC"]

In [ ]:
function coarse_grain(data, resolution)
    @assert length(data) % resolution == 0
    s = length(data) / resolution
    
    data_cs = zeros(resolution)
    for i in 1:resolution
        t = data[Int((i-1)*s+1):Int(i*s)]
        data_cs[i] = mean(t)
    end
    
    return data_cs
end

In [ ]:
coarse_resolution = cr = 32
T_cs = zeros(Nt, coarse_resolution)
for n=1:Nt
    T_cs[n, :] = coarse_grain(T[n, :], coarse_resolution)
end

T_cs = transpose(T_cs) |> gpu;

In [ ]:
T₀ = T_cs[:, 1]
n_train = round(Int, Nt/2)
t_train = t[1:n_train] ./ 86400
tspan_train = (t_train[1], t_train[end])

In [ ]:
#dTdt_NN = Chain(Dense(cr, 2cr, tanh),
#                Dense(2cr, cr))
#cr = 33
ann = Chain(Dense(cr, 2cr, tanh),
            Dense(2cr, 4cr, tanh),
            Dense(4cr, 2cr, tanh),
                Dense(2cr, cr)) |> gpu

n_ann = 2*cr^2 + 2cr + 2*cr^2+cr

#function dTdt_NN(du,u,p,t)
#    du = DiffEqFlux.restructure(ann,p[1:n_ann])(u)
#end

In [12]:
p1 = Flux.data(DiffEqFlux.destructure(ann))
p2 = Float64[1,1]
p3 = param(p1)
ps = Flux.params(p3,T₀)

Params([Float32[0.16553092, -0.20391881, 0.24416095, -0.006131649, 0.038513064, -0.13599229, -0.080458045, 0.104265094, -0.2279427, -0.14112473  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] (tracked)])

In [13]:
cr_z = Nt/cr
#Dzᶠ = 1/cr_z * Tridiagonal(zeros(cr-1), -ones(cr),ones(cr-1))
#Dzᶜ = 1/cr_z * Tridiagonal( -ones(cr-1), ones(cr),zeros(cr-1))
Dzᶠ = Tridiagonal(zeros(cr-1), -ones(cr),ones(cr-1)) |> Array{Float32}
Dzᶠ = Tridiagonal(Dzᶠ)
Dzᶜ = 1/cr_z*Tridiagonal( -ones(cr-1), ones(cr),zeros(cr-1)) |> Array{Float32}
Dzᶜ = Tridiagonal(Dzᶜ)
Dzᶠ[cr, cr] = 0
Dzᶜ[1, 1] = 0
Dzᶠ

32×32 Tridiagonal{Float32,Array{Float32,1}}:
 -1.0   1.0    ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
  0.0  -1.0   1.0    ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅    0.0  -1.0   1.0    ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅    0.0  -1.0   1.0    ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅    0.0  -1.0   1.0       ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅     ⋅    0.0  -1.0  …    ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅    0.0       ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅

In [14]:
#neural_pde_prediction(T₀) = neural_ode(dudt_, T₀, tspan_train, Tsit5(), saveat=t_train, reltol=1e-7, abstol=1e-9)
function dudt_(du,u,p,t)
    #du .= (Dzᶜ*(DiffEqFlux.restructure(ann,p[1:length(p)])(u).*(Dzᶠ*u)))
    du .= (Dzᶜ*(DiffEqFlux.restructure(ann,p[1:length(p)])(u)))
    #for i in 1:length(u)
    #    du[i] = x[i]
    #end
    #du[length(u)] = p[end-1]*u[end] + p[end]*u[end]
    #println("du",du)
end

dudt_ (generic function with 1 method)

In [15]:
#foretold(u,p,t) = DiffEqFlux.restructure(ann,p[1:n_ann])(u)

In [16]:
prob = ODEProblem(dudt_,T₀,tspan_train,p3)
diffeq_adjoint(p3,prob,Tsit5(),u0=T₀,abstol=1e-8,reltol=1e-6)
#solve(prob)

Tracked 32×8 CuArray{Float32,2}:
 19.0156  19.0156  19.0156  19.0156  19.0156  19.0156  19.0156  19.0156
 19.0469  19.0422  19.0328  19.0157  18.9866  18.94    18.8705  18.8602
 19.0781  19.0782  19.0784  19.0787  19.0793  19.0802  19.0816  19.0818
 19.1094  19.113   19.1204  19.1338  19.1567  19.1933  19.2479  19.2559
 19.1406  19.1372  19.1304  19.118   19.0967  19.0628  19.0121  19.0047
 19.1719  19.1734  19.1766  19.1823  19.1922  19.2078  19.2312  19.2346
 19.2031  19.2034  19.204   19.2051  19.2069  19.2099  19.2144  19.215 
 19.2344  19.2315  19.2255  19.2148  19.1964  19.1671  19.1232  19.1168
 19.2656  19.2688  19.2753  19.287   19.307   19.339   19.3868  19.3939
 19.2969  19.2977  19.2994  19.3024  19.3076  19.3159  19.3283  19.3301
 19.3281  19.3278  19.3271  19.326   19.3239  19.3207  19.3159  19.3152
 19.3594  19.3571  19.3526  19.3443  19.3302  19.3078  19.2741  19.2692
 19.3906  19.3912  19.3925  19.3948  19.3987  19.4049  19.4142  19.4155
  ⋮                            

In [17]:
function predict_adjoint()
  diffeq_adjoint(p3,prob,Tsit5(),u0=T₀,saveat=t_train,reltol=1e-6, abstol=1e-8)
end

predict_adjoint (generic function with 1 method)

In [18]:
opt = ADAM()

#data = [(T₀, T_cs[:, 1:n_train])]
data = Iterators.repeated((T₀, T_cs[:, 1:n_train]), 1000)

loss_function(T₀, T_data) = sum(abs2, T_data .- predict_adjoint())

loss_function (generic function with 1 method)

In [19]:
# Callback function to observe training.
cb = function ()
    #println(DiffEqFlux.destructure(ann)[1:10])
    loss = loss_function(T₀, T_cs[:, 1:n_train]) # Not very generalizable...
    println("loss = $loss")
    loss < 100 && Flux.stop()
end

#5 (generic function with 1 method)

In [ ]:
Flux.train!(loss_function, ps, data, opt, cb = cb)

In [47]:
#for _ in 1:20
#    Flux.train!(loss_function, ps, data, opt, cb = cb)
#end

Chain(Dense(32, 64, tanh), Dense(64, 128, tanh), Dense(128, 64, tanh), Dense(64, 32))

In [18]:
tspan = (t[1], t[end]) ./ 86400
#nn_pred = neural_ode(dudt_, T₀, tspan, Tsit5(), saveat=t ./86400, reltol=1e-7, abstol=1e-9) |> Flux.data
prob2 = ODEProblem(dudt_,T₀,tspan,p3)
nn_pred = diffeq_adjoint(p3,prob2,Tsit5(),u0=T₀,abstol=1e-8,reltol=1e-6,saveat=t ./86400) |> Flux.data
z_cs = coarse_grain(z, cr)

anim = @gif for n=1:10:Nt
    t_str = @sprintf("%.2f", t[n] / 86400)
    #println("here1")
    plot(T_cs[:, n], z_cs, linewidth=2,
         xlim=(19, 20), ylim=(-100, 0), label="Data",
         xlabel="Temperature (C)", ylabel="Depth (z)",
         title="Free convection: $t_str days",
         legend=:bottomright, show=false)
    #println("here2")
    if n <= n_train
        plot!(nn_pred[:, n], z_cs, linewidth=2, label="Neural ODE (train)", show=false)
    else
        plot!(nn_pred[:, n], z_cs, linewidth=2, linestyle=:dash, label="Neural ODE (test)", show=false)
    end
end

┌ Info: Saved animation to 
│   fn = C:\Users\daddyj\Documents\6S898-climate-parameterization\notebooks\tmp.gif
└ @ Plots C:\Users\daddyj\.julia\packages\Plots\Iuc9S\src\animation.jl:95


Plots.AnimatedGif("C:\\Users\\daddyj\\Documents\\6S898-climate-parameterization\\notebooks\\tmp.gif")

In [135]:
nn_pred = diffeq_adjoint(p3,prob2,Tsit5(),u0=T₀,abstol=1e-8,reltol=1e-6,saveat=t ./86400) |> Flux.data


32×1153 Array{Float64,2}:
 19.0156  19.0153  19.015   19.0147  …  18.6447  18.6444  18.6441  18.6438
 19.0469  19.0471  19.0472  19.0474     19.2497  19.2498  19.25    19.2502
 19.0781  19.0776  19.0771  19.0766     18.512   18.5115  18.511   18.5106
 19.1094  19.1084  19.1074  19.1065     17.9969  17.9959  17.9949  17.994 
 19.1406  19.1412  19.1418  19.1423     19.7957  19.7962  19.7968  19.7974
 19.1719  19.1724  19.1729  19.1734  …  19.7699  19.7704  19.7709  19.7714
 19.2031  19.2022  19.2013  19.2004     18.166   18.1651  18.1642  18.1633
 19.2344  19.2343  19.2343  19.2342     19.1856  19.1856  19.1856  19.1855
 19.2656  19.2656  19.2655  19.2655     19.1993  19.1992  19.1992  19.1991
 19.2969  19.2973  19.2978  19.2982     19.7993  19.7997  19.8002  19.8006
 19.3281  19.3284  19.3287  19.3289  …  19.6353  19.6356  19.6358  19.6361
 19.3594  19.3586  19.3579  19.3572     18.5278  18.5271  18.5264  18.5256
 19.3906  19.3911  19.3917  19.3922     19.984   19.9845  19.985   19.9856